## Achievable vs perfect search

From the 6/29 phone call, we wanted to know what "perfect" looked like - what can we recover when we look at the original graph with the full query, vs the contracted De Bruijn graph with the MinHash query?

To phrase in terms of our existing [benchmarking document](https://github.com/spacegraphcats/spacegraphcats/blob/master/doc/graph-classification-by-minhash.md):

* B is the original DBG;
* G is the contracted De Bruin graph;
* T is the target subset of the original De Bruijn graph;
* M is the MinHash sketch of T;

the question is:

If we search G for T using M what is our performance? How close can we get to the "perfect" results of searching B (the original DBG) with T directly?

**Important note:** the sensitivity and specifity below are in terms of k-mers in B, not nodes in G. This is different from how we've been doing the other benchmarking; I'll go fix those scripts if that's what we want (which seems likely).

### Labeling the cDBG

In order to compare search results against ground truth, we need to label the nodes in the graph with their origin. That is, we need to know which parts of B belong to the
target subset T.  This is easy in B but requires a bit of work in G, where we have to label
the nodes.

We have two types of nodes in the contracted De Bruijn graph implementation: high degree nodes and linear segments. The high degree nodes are always a single k-mer (and are size 1). Linear segments represent linear regions of the graph and can be size 1 or larger.

CTB's initial labeling of the graph only labeled the high degree nodes, which was easy to implement.  However, these high degree nodes represent only a small portion of B, which is mostly linear. So, more recently, CTB added a `--label-linear-segments` (and also a `--no-label-hdn`) to `walk-dbg.py`, so that we can select whether we label only hdn nodes (default), all nodes (`--label-linear-segments`, or only linear segments `--label-linear-segments --no-label-hdn`) - note that for most purposes, we *don't* want to label linear segments, because it's slow, but for true benchmarking, we need to.


### Running a perfect search on the small test data set

If you search a completely labeled graph with a MinHash sketch that contains all of the k-mers in T, you will get 100% sensitivity and 100% specificity.

In [1]:
!../walk-dbg.py ../data/tr-cross.fa -x 2e7 --label --label-linear-segments 


placing output in directory: tr-cross
gxt will be: tr-cross/tr-cross.gxt
mxt will be: tr-cross/tr-cross.mxt
(note: directory already exists)

building graphs and loading files
finding high degree nodes
(and labeling them, per request)
... find_high_degree_nodes: 10000
... find_high_degree_nodes: 20000
... find_high_degree_nodes: 30000
... find_high_degree_nodes: 40000
... find_high_degree_nodes: 50000
... find_high_degree_nodes: 60000
... find_high_degree_nodes: 70000
... find_high_degree_nodes: 80000
... find_high_degree_nodes: 90000
... find_high_degree_nodes: 10000
... find_high_degree_nodes: 20000
... find_high_degree_nodes: 30000
... find_high_degree_nodes: 40000
... find_high_degree_nodes: 50000
... find_high_degree_nodes: 60000
... find_high_degree_nodes: 70000
... find_high_degree_nodes: 80000
... find_high_degree_nodes: 90000
... find_high_degree_nodes: 100000
traversing linear segments from 131 nodes
... 0 of 131
393 segments, containing 200101 nodes
...doing labeling of lin

In [2]:
# compute a complete signature of what's in tr-1.fa
!../../sourmash/sourmash compute -n 100000 ../data/tr-1.fa -o tr-1.total.sig
!../../sourmash/sourmash dump tr-1.total.sig

# running sourmash subcommand: compute
computing signatures for files: ['../data/tr-1.fa']
Computing signature for ksizes: [31]
... reading sequences from ../data/tr-1.fa
# running sourmash subcommand: dump
loading tr-1.total.sig


In [3]:
# now do the search of the completely labeled De Bruijn graph with the complete signature
!../search-dbg.py -q tr-cross tr-1.total.sig.dump.txt 1

sensitivity: 100.0
specificity: 100.0


### Searching the small test data set with an incomplete signature

What happens if you search with a smaller sketch?  Your specificity stays at 100%, but you don't retrieve all the nodes - this is because in some cases the nodes (especially the high-degree nodes) have no k-mers that are in the smaller sketch.

In [4]:
!../search-dbg.py -q tr-cross ../data/tr-1.fa.sig.dump.txt 1

sensitivity: 98.3
specificity: 100.0


### Trying it out on a larger data set

You see similar results on the larger acido-chunk data set:

In [5]:
!../walk-dbg.py -x 4e9 ../data/acido-chunk?.fa.gz -o acido-chunk --label --label-linear-segments


placing output in directory: acido-chunk
gxt will be: acido-chunk/acido-chunk.gxt
mxt will be: acido-chunk/acido-chunk.mxt
(note: directory already exists)

building graphs and loading files
finding high degree nodes
(and labeling them, per request)
... find_high_degree_nodes: 10000
... find_high_degree_nodes: 20000
... find_high_degree_nodes: 30000
... find_high_degree_nodes: 40000
... find_high_degree_nodes: 50000
... find_high_degree_nodes: 60000
... find_high_degree_nodes: 70000
... find_high_degree_nodes: 80000
... find_high_degree_nodes: 90000
... find_high_degree_nodes: 100000
... find_high_degree_nodes: 110000
... find_high_degree_nodes: 120000
... find_high_degree_nodes: 130000
... find_high_degree_nodes: 140000
... find_high_degree_nodes: 150000
... find_high_degree_nodes: 160000
... find_high_degree_nodes: 170000
... find_high_degree_nodes: 180000
... find_high_degree_nodes: 190000
... find_high_degree_nodes: 200000
... find_high_degree_nodes: 210000
... find_high_degree_no

In [6]:
!../search-dbg.py -q acido-chunk ../data/acido-chunk5.fa.sig.dump.txt 5

sensitivity: 99.2
specificity: 100.0
